In [3]:
import requests
import pandas as pd
import json


In [4]:
# Suppress SSL warning
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [19]:
#Query the Flow Auditor 
req = {
    "filter": {
     "operator": "or",
     "rules": [ 
                  {
            "field": "proto",
            "operand":"TCP",
            "operator":"=",
         },
                 {
            "field": "proto",
            "operand":"UDP",
            "operator":"=",
         }
     ]},
       
    "from": "-1d",
    "types": ["ns-traffic-monitor"], #<--Our INGRESS/EGRESS Record Store!
    "limit": 1000
}
req_body = json.dumps(req)
#req_body

In [12]:
#Get EGRESS/INGRESS records
endpoint = 'https://RevealX-APPLIANCE/api/v1/records/search'
headers = {'Accept': 'application/json', 'Authorization': 'ExtraHop apikey=YOUR-API-KEY'}
r = requests.post(endpoint, headers=headers, data=req_body, verify=False)

In [18]:
# see response
#r.json()

In [14]:
# transform response into list of tuples
records = [(rec['_type'],
         rec['_source']['timestamp'],
         rec['_source']['clientAddr']['value'], 
         rec['_source']['serverAddr']['value'],
         rec['_source']['country']) for rec in r.json()['records']]

In [15]:
# create pandas dataframe from list of tuples
records_df = pd.DataFrame.from_records(records, columns=["type", "timestamp", "clientIP", "Server Address","country"])
records_df.timestamp = pd.to_datetime(records_df.timestamp, unit='ms')
#records_df

In [16]:
#Source the location of your Shodan JSON file
data = [json.loads(line) for line in open('/Users/johnsmith/Downloads/shodan-export.json', 'r')]
shodan_df = pd.DataFrame(data)
#shodan_df

In [17]:
#Compare the two dataframes to find sessions/transactions with known bad hosts
joined_ips=pd.merge(records_df,shodan_df, right_on="ip_str", left_on='Server Address')
joined_ips


,type,timestamp_x,clientIP,Server Address,country,_shodan,asn,data,domains,hash,...,opts,org,os,port,product,tags,timestamp_y,transport,upnp,version
0,ns-traffic-monitor,2021-01-07 23:34:22.146604004,172.16.178.85,124.161.241.225,China,{u'crawler': u'c4f6dbc45caa04f64112a93cf20c797...,AS4837,HTTP/1.1 200 OK \r\nContent-Type: text/html\r\...,[],-294857937,...,{},China Unicom Liaoning,None,10000,Cobalt Strike team server,NaN,2021-01-05T05:36:27.396572,tcp,NaN,NaN
1,ns-traffic-monitor,2021-01-07 23:33:40.927437988,172.16.178.85,124.161.241.225,China,{u'crawler': u'c4f6dbc45caa04f64112a93cf20c797...,AS4837,HTTP/1.1 200 OK \r\nContent-Type: text/html\r\...,[],-294857937,...,{},China Unicom Liaoning,None,10000,Cobalt Strike team server,NaN,2021-01-05T05:36:27.396572,tcp,NaN,NaN
2,ns-traffic-monitor,2021-01-07 21:25:39.452676025,172.16.178.85,123.240.66.68,Taiwan,{u'crawler': u'264b5a9d15a64f96a4768e9d8081b9d...,AS131596,HTTP/1.1 200 OK \r\nContent-Type: text/html\r\...,[tbcnet.net.tw],-1564662150,...,{},TBC,None,10000,Cobalt Strike team server,NaN,2021-01-05T06:33:35.951141,tcp,NaN,NaN
